In [1]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np
from datasets import DatasetDict
import torch
import numpy as np
from torchvision.transforms import Resize, Compose, Normalize

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer, LightningModule
from torchvision import models
import torch.nn.functional as F
from torchmetrics import Accuracy
from sklearn.utils.class_weight import compute_class_weight
import wandb

# Modeling
import torch
import torch.nn as nn
import numpy as np
from torchvision.transforms.functional import pil_to_tensor
from torch.utils.data import Dataset,DataLoader

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline

print(f'PyTorch version {torch.__version__}')
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
    device = 'cuda'
else:
    print("GPU is not available.")
    device = 'cpu'

PyTorch version 2.2.1+cu121
GPU is available: NVIDIA GeForce RTX 4070 Laptop GPU


In [2]:
wandb.init(entity='vincenzo-timmel')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vincenzo-timmel. Use `wandb login --relogin` to force relogin


In [3]:
ds = load_dataset('i4ds/radio-sunburst-ecallisto')

In [4]:
dd = DatasetDict()
dd['train'] = ds['train']
dd['test'] = ds['test']
dd['validation'] = ds['validation']

In [5]:
def to_torch_tensor(example):
    # Convert the PIL image to a tensor
    example['image'] = pil_to_tensor(example['image'])
    return example

def scale(example, max_value=255):
    example['image'] = torch.div(example['image'], max_value)
    return example

def preprocess(example):
    example = to_torch_tensor(example)
    example = scale(example)
    return example

In [6]:
# Dataset
class EcallistoData(Dataset):
    def __init__(self, dataset, prepro=preprocess, transform=None, return_all_columns=False):
        super().__init__()
        self.data = dataset
        self.prepro = prepro
        self.transform = transform
        self.return_all_columns = return_all_columns

    def __len__(self):
        """Function to return the number of records in the dataset
        """ 
        return len(self.data)
    
    def __getitem__(self, index):
        """Function to return samples corresponding to a given index from a dataset
        """ 
        # Transform images
        if self.prepro:
            img = self.prepro(self.data[index])
        if self.transform:
            img['image'] = self.transform(img['image'])
        if not self.return_all_columns:
            return torch.tensor(img['image']), torch.tensor(self.data[index]['label'])
        else: 
            return self.data[index]

In [7]:
normalize = Normalize(mean=0.5721, std=0.1100)
size = (224, 244)

In [8]:
_transforms = Compose([Resize(size), normalize])

In [9]:
ds_train = EcallistoData(dd['train'], transform=_transforms)
ds_valid = EcallistoData(dd['validation'], transform=_transforms)
ds_test = EcallistoData(dd['test'], return_all_columns=True)

In [17]:
BATCH_SIZE = 32

In [30]:
train_dataloader = DataLoader(ds_train,
                    batch_size=BATCH_SIZE,
                    num_workers=0,
                    shuffle=True)

val_dataloader = DataLoader(ds_valid,
                    batch_size=BATCH_SIZE,
                    num_workers=0,
                    shuffle=False)

In [31]:
wandb_logger = WandbLogger(log_model="all")

In [36]:
class EfficientNet(LightningModule):
  def __init__(self, n_classes, class_weights, model_weights=None):
    super().__init__()
    self.efficient_net = models.efficientnet_v2_s(weights=model_weights)
    # Adapt layer to our layer
    self.efficient_net.classifier[1] = nn.Linear(in_features=1280, out_features=n_classes, bias=True)
    self.accuracy = Accuracy(task="multiclass", num_classes=n_classes)
    self.class_weights = class_weights
    
  def forward(self,x):
    # Convert grayscale image to 3-channel image if it's not already
    if x.size(1) == 1:
      x = x.repeat(1, 3, 1, 1)
    out = self.efficient_net(x)
    return out
  
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(),lr = 1e-4)
    return optimizer
  
  def training_step(self,batch,batch_idx):
    x,y = batch
    y_hat = self(x)
    if self.class_weights is not None:
        loss = F.cross_entropy(y_hat, y, weight=self.class_weights.to(y.device))
    else:
        loss = F.cross_entropy(y_hat, y)
    # logs metrics for each training_step - [default:True],
    # the average across the epoch, to the progress bar and logger-[default:False]
    acc = self.accuracy(y_hat,y)
    self.log("train_acc",acc,on_step=True,on_epoch=True,prog_bar=True,logger=True),
    self.log("train_loss",loss,on_step=True,on_epoch=True,prog_bar=True,logger=True)
    return loss
  
  def validation_step(self,batch,batch_idx):
    x,y = batch
    y_hat = self(x)
    loss = F.cross_entropy(y_hat,y)
    acc = self.accuracy(y_hat,y)
    # logs metrics for each validation_step - [default:False]
    #the average across the epoch - [default:True]
    self.log("val_acc",acc,prog_bar=True,logger=True),
    self.log("val_loss",loss,prog_bar=True,logger=True)

In [37]:
cw = compute_class_weight(class_weight='balanced', classes=np.unique(dd['train']['label']), y=dd['train']['label'])

In [38]:
model = EfficientNet(len(np.unique(dd['train']['label'])), torch.tensor(cw, dtype=torch.float))

In [39]:
trainer = Trainer(accelerator='gpu', 
    max_epochs=1, logger=wandb_logger)
trainer.fit(model=model, 
            train_dataloaders=train_dataloader,
              val_dataloaders=val_dataloader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\vince\miniconda3\envs\flaresense-v2\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:652: Checkpoint directory .\lightning_logs\5jm8mxp6\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name          | Type               | Params
-----------------------------------------------------
0 | efficient_net | EfficientNet       | 20.2 M
1 | accuracy      | MulticlassAccuracy | 0     
-----------------------------------------------------
20.2 M    Trainable params
0         Non-trainable params
20.2 M    Total params
80.746    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\vince\miniconda3\envs\flaresense-v2\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
C:\Users\vince\AppData\Local\Temp\ipykernel_29208\230362761.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(img['image']), torch.tensor(self.data[index]['label'])
c:\Users\vince\miniconda3\envs\flaresense-v2\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

c:\Users\vince\miniconda3\envs\flaresense-v2\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
